In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import joblib
from pathlib import Path
import re
import json, warnings
warnings.filterwarnings("ignore")

In [ ]:
# === 1) Fuente de datos ===

# Intenta cargar df_modelo desde varios candidatos
def load_df_modelo():
    candidates = [
        Path("artifacts/_cache/df_modelo.parquet"),
        Path("artifacts/_cache/df_modelo.csv"),
        Path("Data/df_modelo.parquet"),
        Path("Data/df_modelo.csv"),
    ]
    for p in candidates:
        if p.exists():
            print(f"Cargando df_modelo desde: {p}")
            if p.suffix == ".parquet":
                return pd.read_parquet(p)
            return pd.read_csv(p)
    raise FileNotFoundError(
        "No encontré df_modelo en disco. "
    )

df_raw = load_df_modelo()


# === 2) Normalización de nombres de columnas ===
def norm(s):
    s = str(s).strip()
    s = s.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
    s = re.sub(r"[^a-zA-Z0-9_]", "", s)
    return s.lower()

ren_map = {c: norm(c) for c in df_raw.columns}
df = df_raw.rename(columns=ren_map)

# Alias hacia nombres estándar (sin acentos y camel-case)
ALIASES = {
    "fecha": ["fecha","date","periodo","mes","yyyymm"],
    "rendimiento": ["rendimiento","yield","t_ha","tha"],
    "ndvi": ["ndvi"],
    "evi": ["evi"],
    "Precipitacion": ["precipitacion","ppt","lluvia","rain"],
    "TempMax": ["tempmax","tmax","temp_max"],
    "TempMin": ["tempmin","tmin","temp_min"],
    "HumedadRelativa": ["humedadrelativa","hr","humedad","rh"]
}

def map_to_std(df, aliases):
    out = {}
    for std, cands in aliases.items():
        for c in cands:
            if c in df.columns:
                out[std] = c
                break
    return out

colmap = map_to_std(df, ALIASES)
# Promociona a nombres estándar
for std, src in colmap.items():
    if std != src:
        df[std] = df[src]

# === 3) Validaciones mínimas ===
req_min = {"fecha","rendimiento"}
assert req_min.issubset(df.columns), f"Faltan columnas mínimas {req_min - set(df.columns)}"

# === 4) Preparación de fecha y estacionalidad ===
if "fecha" in df.columns:
    # si vino yyyymm en la misma columna ya mapeada a 'fecha'
    # intenta parsear primero como YYYY-MM
    df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce", format="%Y-%m")
    # si quedó NaT, intenta parseo genérico
    nan_mask = df["fecha"].isna()
    if nan_mask.any():
        df.loc[nan_mask, "fecha"] = pd.to_datetime(df.loc[nan_mask, "fecha"], errors="coerce")
else:
    raise ValueError("No se pudo construir 'fecha'.")

df["rendimiento"] = pd.to_numeric(df["rendimiento"], errors="coerce")

df["month"] = df["fecha"].dt.month
df["month_sin"] = np.sin(2*np.pi*df["month"]/12.0)
df["month_cos"] = np.cos(2*np.pi*df["month"]/12.0)

# === 5) Selección de columnas finales (las que existan) ===
CAND = ["NDVI","EVI","Precipitacion","TempMax","TempMin","HumedadRelativa","month_sin","month_cos"]
present = []
for c in CAND:
    # están con camel-case exacto; crea si existe en lower estándar
    if c in df.columns:
        present.append(c)
    elif c.lower() in df.columns:
        df[c] = pd.to_numeric(df[c.lower()], errors="coerce")
        present.append(c)

# Garantizar tipos numéricos
for c in present:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Dataset de salida para tablero (orden recomendado)
cols_out = ["fecha","rendimiento"] + present
df_out = df[cols_out].sort_values("fecha").reset_index(drop=True)

# === 6) Guardar dataset_modelo.csv canónico ===
ARTI = Path("artifacts"); ARTI.mkdir(exist_ok=True)
df_out.to_csv(ARTI / "dataset_modelo.csv", index=False)

print("Dataset_modelo.csv reconstruido en ./artifacts/")
print("Columnas:", df_out.columns.tolist())
print("Filas:", len(df_out))


Cargando df_modelo desde: artifacts\_cache\df_modelo.parquet
Dataset_modelo.csv reconstruido en ./artifacts/
Columnas: ['fecha', 'rendimiento', 'NDVI', 'EVI', 'Precipitacion', 'TempMax', 'TempMin', 'HumedadRelativa', 'month_sin', 'month_cos']
Filas: 96


In [10]:
# === 1) Cargar SIEMPRE el dataset recién construido ===
ARTI = Path("artifacts"); ARTI.mkdir(exist_ok=True)
df = pd.read_csv(ARTI / "dataset_modelo.csv", parse_dates=["fecha"])

assert {"fecha","rendimiento"}.issubset(df.columns)
y = df["rendimiento"].astype(float)

CAND = ["NDVI","EVI","Precipitacion","TempMax","TempMin","HumedadRelativa","month_sin","month_cos"]
feature_cols = [c for c in CAND if c in df.columns]
X = df[feature_cols].copy()
for c in feature_cols:
    X[c] = pd.to_numeric(X[c], errors="coerce")

# === 2) Split temporal (sin shuffle) + candidatos de modelo ===
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, shuffle=False)

prepro = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler" , StandardScaler())
])

candidatos = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Regresión Lineal": LinearRegression(),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
}

def evalua(nombre, est):
    pipe = Pipeline([("prepro", prepro), ("model", est)])
    pipe.fit(X_tr, y_tr)
    pred = pipe.predict(X_te)
    mae = mean_absolute_error(y_te, pred)
    rmse = np.sqrt(mean_squared_error(y_te, pred))
    r2 = r2_score(y_te, pred)
    return {"MODELO": nombre, "MAE": mae, "RMSE": rmse, "R2": r2, "pipeline": pipe}

res = [evalua(n, m) for n, m in candidatos.items()]
df_resultados = pd.DataFrame([{k:v for k,v in r.items() if k!="pipeline"} for r in res]).sort_values("R2", ascending=False)

best_name = df_resultados.iloc[0]["MODELO"]
best_pipe = [r["pipeline"] for r in res if r["MODELO"] == best_name][0]

# === 3) Reentrenar en TODO el histórico con el mejor modelo ===
pipe_final = Pipeline([("prepro", prepro), ("model", candidatos[best_name])])
pipe_final.fit(X, y)

# === 4) Guardar artefactos (sobrescribe los viejos) ===
joblib.dump(pipe_final, ARTI / "modelo_boyaca.pkl")
joblib.dump(feature_cols, ARTI / "feature_cols.pkl")

meta = {
    "model": type(pipe_final.named_steps["model"]).__name__,
    "n_obs": int(len(y)),
    "features": feature_cols,
    "metricas_test_snapshot": {
        "R2":   float(df_resultados.iloc[0]["R2"]),
        "RMSE": float(df_resultados.iloc[0]["RMSE"]),
        "MAE":  float(df_resultados.iloc[0]["MAE"]),
    },
    "source": "artifacts/dataset_modelo.csv"
}
with open(ARTI / "metadata.json","w",encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)

# (opcional) Deja a mano estos CSV para el tablero
df.to_csv(ARTI / "dataset_modelo.csv", index=False)          # ya es el mismo
df_resultados.to_csv(ARTI / "df_resultados.csv", index=False)

print("✅ Artefactos actualizados en ./artifacts/:")
print("   - modelo_boyaca.pkl")
print("   - feature_cols.pkl")
print("   - metadata.json")
print("   - dataset_modelo.csv")
print("   - df_resultados.csv")

✅ Artefactos actualizados en ./artifacts/:
   - modelo_boyaca.pkl
   - feature_cols.pkl
   - metadata.json
   - dataset_modelo.csv
   - df_resultados.csv
